In [248]:
#必要なライブラリをインポート
import pandas as pd
import numpy as np
from IPython.display import display

df = pd.read_csv('csv/chiyoda_chuo_taito.csv', sep='\t', encoding='UTF-8')

In [249]:
# 不要な列を削除
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [250]:
# df[df['間取り'] in ['1LDK', '2DK', '2LDK', '2SLDK', '3K', '3DK', '3LDK', '3SLDK']] がエラー出てびびった
df = df[df['間取り'].isin(['1LDK', '2DK', '2LDK', '2SLDK', '3K', '3DK', '3LDK', '3SLDK'])]

In [251]:
#その他費用を、「敷金」「礼金」「保証金」「敷引,償却」に分割
splitted1 = df['敷/礼/保証/敷引,償却'].str.split('/', expand=True)
splitted1.columns = ['敷金', '礼金', '保証金', '敷引,償却']

In [252]:
df = pd.concat([df, splitted1], axis=1)
df.head()

,マンション名,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積,URL,敷金,礼金,保証金,"敷引,償却"
0,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,13階,20.5万円,-,20.5万円/20.5万円/-/-,1LDK,42.04m,https://suumo.jp/chintai/jnc_000041057199/?bc=...,20.5万円,20.5万円,-,-
1,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,30階,22万円,-,44万円/22万円/-/-,1LDK,58.25m,https://suumo.jp/chintai/jnc_000041057199/?bc=...,44万円,22万円,-,-
2,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,39階,24万円,-,24万円/24万円/-/-,1LDK,49.95m,https://suumo.jp/chintai/jnc_000041057199/?bc=...,24万円,24万円,-,-
4,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,22階,24.5万円,20000円,24.5万円/24.5万円/-/-,2LDK,58.94m,https://suumo.jp/chintai/jnc_000041057199/?bc=...,24.5万円,24.5万円,-,-
5,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,20階,26万円,20000円,26万円/26万円/-/-,2LDK,68.62m,https://suumo.jp/chintai/jnc_000041057199/?bc=...,26万円,26万円,-,-


In [253]:
#数値として扱いたいので、不要な文字列を削除
df['賃料'] = df['賃料'].str.replace(u'万円', u'')
df['敷金'] = df['敷金'].str.replace(u'万円', u'')
df['礼金'] = df['礼金'].str.replace(u'万円', u'')
df['保証金'] = df['保証金'].str.replace(u'万円', u'')
df['管理費'] = df['管理費'].str.replace(u'円', u'')
df['築年数'] = df['築年数'].str.replace(u'新築', u'0') #新築は築年数0年とする
df['築年数'] = df['築年数'].str.replace(u'築', u'')
df['築年数'] = df['築年数'].str.replace(u'年', u'')
df['専有面積'] = df['専有面積'].str.replace(u'm', u'')

In [254]:
#「-」を0に変換
df['管理費'] = df['管理費'].replace('-',0)
df['敷金'] = df['敷金'].replace('-',0)
df['礼金'] = df['礼金'].replace('-',0)
df['保証金'] = df['保証金'].replace('-',0)

In [255]:
#文字列から数値に変換
df['賃料'] = pd.to_numeric(df['賃料'])
df['管理費'] = pd.to_numeric(df['管理費'])
df['敷金'] = pd.to_numeric(df['敷金'])
df['礼金'] = pd.to_numeric(df['礼金'])
df['保証金'] = pd.to_numeric(df['保証金'])
df['築年数'] = pd.to_numeric(df['築年数'])
df['専有面積'] = pd.to_numeric(df['専有面積'])

In [256]:
#単位を合わせるために、管理費以外を10000倍。
df['賃料'] = df['賃料'] * 10000
df['敷金'] = df['敷金'] * 10000
df['礼金'] = df['礼金'] * 10000
df['保証金'] = df['保証金'] * 10000

In [257]:
#管理費は実質的には賃料と同じく毎月支払うことになるため、「賃料+管理費」を家賃を見る指標とする
df['賃料+管理費'] = df['賃料'] + df['管理費']

#敷金/礼金と保証金は同じく初期費用であり、どちらかが適用されるため、合計を初期費用を見る指標とする
df['初期費用（敷/礼/保証金）'] = df['敷金'] + df['礼金'] + df['保証金']

In [258]:
#住所を「東京都」「〜区」「市町村番地」に分割
splitted6 = df['住所'].str.split('区', expand=True)
splitted6.columns = ['区', '市町村']
splitted6['区'] = splitted6['区'] + '区'
splitted6['区'] = splitted6['区'].str.replace('東京都','')

In [259]:
#階を数値化。地下はマイナスとして扱う
splitted10 = df['階'].str.split('-', expand=True)
splitted10.columns = ['階1', '階2']
splitted10['階1'].str.encode('cp932')
splitted10['階1'] = splitted10['階1'].str.replace(u'階', u'')
splitted10['階1'] = splitted10['階1'].str.replace(u'B', u'-')
splitted10['階1'] = pd.to_numeric(splitted10['階1'])
df = pd.concat([df, splitted10], axis=1)

In [260]:
#建物高さを数値化。地下は無視。
df['建物高さ'].str.encode('cp932')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下1地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下2地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下3地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下4地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下5地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下6地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下7地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下8地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下9地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'平屋', u'1')
df['建物高さ'] = df['建物高さ'].str.replace(u'階建', u'')
df['建物高さ'] = pd.to_numeric(df['建物高さ'])

In [261]:
# 不要な列を削除
df.drop(['階2'], axis=1, inplace=True)

In [262]:
#カラムを入れ替えて、csvファイルとして出力
# df[['マンション名','住所','間取り','築年数','建物高さ','階1','専有面積','賃料+管理費','敷/礼/保証金',
#                '賃料','管理費','敷金','礼金','保証金', 'URL']]

# df.to_csv('all.csv', sep = '\t',encoding='UTF-8')

In [263]:
# df['平米あたりの賃料'] = df.applymap(lambda row: int(row['専有面積']) / int(row['']))
df['平米あたりの賃料'] = df['賃料+管理費'] / df['専有面積']

In [264]:
df['平米あたりの賃料'] = round(df['平米あたりの賃料'])

In [271]:
df.sort_values(by = '平米あたりの賃料', inplace = True)

In [272]:
df.columns

Index(['マンション名', '住所', '立地1', '立地2', '立地3', '築年数', '建物高さ', '階', '賃料', '管理費',
       '敷/礼/保証/敷引,償却', '間取り', '専有面積', 'URL', '敷金', '礼金', '保証金', '敷引,償却',
       '賃料+管理費', '初期費用（敷/礼/保証金）', '階1', '平米あたりの賃料'],
      dtype='object')

In [270]:
df.head()

,マンション名,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,...,専有面積,URL,敷金,礼金,保証金,"敷引,償却",賃料+管理費,初期費用（敷/礼/保証金）,階1,平米あたりの賃料
10305,村田ビル,東京都台東区橋場２,ＪＲ常磐線/南千住駅 歩15分,つくばエクスプレス/南千住駅 歩15分,東京メトロ日比谷線/南千住駅 歩15分,NaN,NaN,3階,130000.0,0,...,82.62,https://suumo.jp/chintai/jnc_000013372167/?bc=...,130000.0,130000.0,0.0,-,130000.0,260000.0,3,1573.0
10296,東京メトロ銀座線 浅草駅 9階建 築35年,東京都台東区浅草６,東京メトロ銀座線/浅草駅 歩7分,都営浅草線/浅草駅 歩11分,東武伊勢崎線/浅草駅 歩6分,NaN,NaN,3階,130000.0,0,...,82.62,https://suumo.jp/chintai/jnc_000039756658/?bc=...,130000.0,130000.0,0.0,-,130000.0,260000.0,3,1573.0
10238,ロイヤルマンション,東京都台東区西浅草３,つくばエクスプレス/浅草駅 歩3分,東武伊勢崎線/浅草駅 歩10分,東京メトロ銀座線/田原町駅 歩10分,NaN,NaN,4-6階,126000.0,9000,...,85.00,https://suumo.jp/chintai/jnc_000026240591/?bc=...,126000.0,0.0,0.0,12.6万円,135000.0,126000.0,4,1588.0
8433,東京メトロ日比谷線 八丁堀駅 10階建 築13年,東京都中央区新川２,東京メトロ日比谷線/八丁堀駅 歩4分,東京メトロ東西線/茅場町駅 歩7分,東京メトロ半蔵門線/水天宮前駅 歩11分,30.0,5.0,1階,80000.0,0,...,50.00,https://suumo.jp/chintai/jnc_000037549964/?bc=...,80000.0,80000.0,0.0,-,80000.0,160000.0,1,1600.0
10288,ダイアパレス水道橋,東京都千代田区神田三崎町３,ＪＲ中央線/水道橋駅 歩3分,都営三田線/水道橋駅 歩7分,NaN,NaN,NaN,2-3階,98000.0,0,...,60.56,https://suumo.jp/chintai/jnc_000041028009/?bc=...,98000.0,196000.0,0.0,-,98000.0,294000.0,2,1618.0
